### Grab Data for All 2021 Qualifying Sessions

I'll try to grab telemetry data for every driver's qualifying session in 2021 here as a starting point for further analysis

In [1]:
#Import necessary packages
import fastf1 as ff1
import numpy as np
import pandas as pd

#Enable the cache
ff1.Cache.enable_cache('C:/Users/jackh/OneDrive/Documents/Python Scripts/f1_cache') 

In [2]:
gp_list = range(1,23)

test = ff1.get_session(2021, 5, 'Q')

In [3]:
#Create a nested for loop that runs through the fastest qualifying lap for every driver & every qualifying session in the 2021 season. 
# #The loop starts with drivers & iterates through GPs, rather than starting with a full GP list to iterate through. 
# #I'm not sure why this works & the other way doesn't, but perhaps it is to eliminate NAs for drivers that didn't participate in a qualifying.
driver_team = pd.DataFrame({'Driver':['VER', 'PER', 'HAM', 'BOT', 'LEC', 'SAI', 'RIC', 'NOR', 'ALO', 'OCO'
                                      , 'VET', 'STR', 'GAS', 'TSU', 'RUS', 'LAT', 'RAI', 'GIO', 'MSC', 'MAZ'],
                            'Team': ['Red Bull', 'Red Bull', 'Mercedes', 'Mercedes', 'Ferrari', 'Ferrari', 'McLaren', 'McLaren', 'Alpine', 'Alpine'
                                     , 'Aston Martin', 'Aston Martin', 'Alpha Tauri', 'Alpha Tauri', 'Williams', 'Williams', 'Alfa Romeo' , 'Alfa Romeo', 'Haas', 'Haas']})

driver_list = driver_team['Driver'].tolist()

In [4]:

sessions = pd.DataFrame([])
df = pd.DataFrame([])

for drv in driver_list:
    
    for i in gp_list:
        quali = ff1.get_session(2021, i, 'Q')
        laps = quali.load_laps(with_telemetry=True)
        #Identify the fastest lap for each driver
        fastest = laps.pick_driver(drv)
        #Occasionally there will be times where a driver does not set a lap. The below looks for these cases & removes them
        fastest = fastest[fastest['LapTime'].notnull()]
        if fastest.empty:
            continue
        else:
            fastest = fastest.pick_fastest().get_telemetry().add_distance()
            #And add the driver variable
            fastest['Driver'] = drv
            #Add ID for which Grand Prix this data is for
            fastest['GP'] = i
            fastest = pd.DataFrame(fastest)
            #Finally, add this data frame to the table for all drivers
            df = df.append(fastest)
    
    sessions = sessions.append(df)

core           INFO 	Loading laps for Bahrain Grand Prix - Qualifying [v2.1.13]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 20 drivers: ['10', '63', '11', '47', '9', '3', '31', '77', '5', '7', '55', '33', '22', '99', '16', '4', '18', '14', '6', '44']
core           INFO 	Loading laps for Emilia Romagna Grand Prix - Qualifying [v2.1.13]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api

In [6]:
#Validate that the data is working

print(sessions['Driver'].unique())

print(sessions['GP'].unique())

sessions.head()

['VER' 'PER' 'HAM' 'BOT' 'LEC' 'SAI' 'RIC' 'NOR' 'ALO' 'OCO' 'VET' 'STR'
 'GAS' 'TSU' 'RUS' 'LAT' 'RAI' 'GIO' 'MSC' 'MAZ']
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,RelativeDistance,Status,X,Y,Z,Distance,Driver,GP
2,2021-03-27 15:59:07.466,0 days 01:14:07.081000,,715.495556,0 days 00:00:00,10531,295,8,100,0,12,interpolation,0.000016,OnTrack,-384,1200,-159,0.000000,VER,1
3,2021-03-27 15:59:07.603,0 days 01:14:07.218000,,715.495556,0 days 00:00:00.137000,10537,296,8,100,0,12,pos,0.002104,OnTrack,-378,1312,-159,11.264444,VER,1
4,2021-03-27 15:59:07.635,0 days 01:14:07.250000,,715.495556,0 days 00:00:00.169000,10549,297,8,100,0,12,car,0.002592,OnTrack,-376,1338,-159,13.904444,VER,1
5,2021-03-27 15:59:07.823,0 days 01:14:07.438000,,715.495556,0 days 00:00:00.357000,10635,297,8,100,0,12,pos,0.005465,OnTrack,-369,1493,-159,29.414444,VER,1
6,2021-03-27 15:59:07.875,0 days 01:14:07.490000,44,715.495556,0 days 00:00:00.409000,10721,298,8,100,0,12,car,0.006262,OnTrack,-367,1536,-158,33.718889,VER,1


In [7]:
#List of GPs to numbers
gp_location = pd.DataFrame({'GP':gp_list,
                            'Location': ['Bahrain', 'Emilia-Romagna', 'Portugal', 'Spain', 'Monaco'
                                         , 'Azerbaijan', 'France', 'Austria1', 'Austria2', 'Great Britain'
                                         , 'Hungary', 'Belgium', 'Netherlands', 'Italy', 'Russia'
                                         , 'Turkey', 'USA', 'Mexico', 'Brazil', 'Qatar'
                                         , 'Saudi Arabia', 'Abu Dhabi']})

#merge with main df
final = sessions.merge(gp_location, how = 'inner', on = 'GP')

In [8]:
final.head()

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,...,Source,RelativeDistance,Status,X,Y,Z,Distance,Driver,GP,Location
0,2021-03-27 15:59:07.466,0 days 01:14:07.081000,,715.495556,0 days 00:00:00,10531,295,8,100,0,...,interpolation,0.000016,OnTrack,-384,1200,-159,0.000000,VER,1,Bahrain
1,2021-03-27 15:59:07.603,0 days 01:14:07.218000,,715.495556,0 days 00:00:00.137000,10537,296,8,100,0,...,pos,0.002104,OnTrack,-378,1312,-159,11.264444,VER,1,Bahrain
2,2021-03-27 15:59:07.635,0 days 01:14:07.250000,,715.495556,0 days 00:00:00.169000,10549,297,8,100,0,...,car,0.002592,OnTrack,-376,1338,-159,13.904444,VER,1,Bahrain
3,2021-03-27 15:59:07.823,0 days 01:14:07.438000,,715.495556,0 days 00:00:00.357000,10635,297,8,100,0,...,pos,0.005465,OnTrack,-369,1493,-159,29.414444,VER,1,Bahrain
4,2021-03-27 15:59:07.875,0 days 01:14:07.490000,44,715.495556,0 days 00:00:00.409000,10721,298,8,100,0,...,car,0.006262,OnTrack,-367,1536,-158,33.718889,VER,1,Bahrain


In [9]:
#Write output to CSV
final.to_csv('C:/Users/jackh/OneDrive/Documents/Python Scripts/2021_f1_qualifying_laps.csv')